# Combining & Merging datasets

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Data contained in pandas objects can be combined together in a number of ways:

1. [**Database-Style DataFrame Joins**](#Database-Style-DataFrame-Joins)

    - [**Merge**](#Merge) [`pd.merge(df1, df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge): for combining data on common columns or indices

    - [**Join**](#Join) [`df1.join(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join): for combining data on a key column or an index
    

2. [**Concatenating Along an Axis**](#Concatenating-Along-an-Axis)

    - [**Concat**](#Concat) [`pd.concat([df1, df2])`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat): for combining DataFrames across rows or columns

    - [**Append**](#Append) [`df1.append(df2)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html): stacking vertically

------------------------------------------------------------------------------------------------------------------------------

## Database-Style DataFrame Joins

# Merge

Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on. When performing a cross merge, no column specifications to merge on are allowed.

When you use merge(), you’ll provide two required arguments:

- The left DataFrame
- The right DataFrame

After that, you can provide a number of optional arguments to define how your datasets are merged:

**how**: This defines what kind of merge to make. It defaults to 'inner', but other possible options include 'outer', 'left', and 'right'.

**on**: Use this to tell merge() which columns or indices (also called key columns or key indices) you want to join on. This is optional. If it isn’t specified, and left_index and right_index (covered below) are False, then columns from the two DataFrames that share names will be used as join keys. If you use on, then the column or index you specify must be present in both objects.

**left_on and right_on**: Use either of these to specify a column or index that is present only in the left or right objects that you are merging. Both default to None.

**left_index and right_index**: Set these to True to use the index of the left or right objects to be merged. Both default to False.

**suffixes**: This is a tuple of strings to append to identical column names that are not merge keys. This allows you to keep track of the origins of columns with the same name.

### Joining (Merging) DataFrames

Using the [MovieLens 100k data](http://grouplens.org/datasets/movielens/), let's create two DataFrames:

- **movies**: shows information about movies, namely a unique **movie_id** and its **title**
- **ratings**: shows the **rating** that a particular **user_id** gave to a particular **movie_id** at a particular **timestamp**

In [ ]:
#first things first
import pandas as pd
import numpy as np

#### Movies

In [2]:
url = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.item"

In [8]:
movie_cols = ['movie_id', 'title']
movies = pd.read_csv(url, sep='|', header=None, names=movie_cols, usecols=[0, 1])
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [9]:
movies.shape

(1682, 2)

In [10]:
movies.movie_id.nunique()

1682

#### Ratings

In [15]:
url1 = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.data"

In [16]:
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(url1, sep='\t', header=None, names=rating_cols)

In [24]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [19]:
ratings.shape

(100000, 4)

In [20]:
ratings.movie_id.nunique()

1682

In [25]:
ratings.loc[ratings.movie_id == 1, :]

,user_id,movie_id,rating,timestamp
24,308,1,4,887736532
454,287,1,5,875334088
957,148,1,4,877019411
971,280,1,4,891700426
1324,66,1,3,883601324
...,...,...,...,...
99423,895,1,4,879437950
99445,747,1,5,888639138
99631,786,1,4,882841828
99704,800,1,4,887646283


#### Merging Movies and Ratings

Let's pretend that you want to examine the ratings DataFrame, but you want to know the **title** of each movie rather than its **movie_id**. The best way to accomplish this objective is by "joining" (or "merging") the DataFrames using the Pandas `merge` function:

In [26]:
movies.shape

(1682, 2)

In [27]:
ratings.shape

(100000, 4)

In [28]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp'], dtype='object')

In [29]:
movies.columns

Index(['movie_id', 'title'], dtype='object')

In [31]:
movie_ratings = pd.merge(movies, ratings)
movie_ratings.columns

Index(['movie_id', 'title', 'user_id', 'rating', 'timestamp'], dtype='object')

In [36]:
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [34]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [32]:
movie_ratings.head()

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


In [33]:
movie_ratings.shape

(100000, 5)

Here's what just happened:

- Pandas noticed that movies and ratings had one column in common, namely **movie_id**. This is the "key" on which the DataFrames will be joined.
- The first **movie_id** in movies is 1. Thus, Pandas looked through every row in the ratings DataFrame, searching for a movie_id of 1. Every time it found such a row, it recorded the **user_id**, **rating**, and **timestamp** listed in that row. In this case, it found 452 matching rows.
- The second **movie_id** in movies is 2. Again, Pandas did a search of ratings and found 131 matching rows.
- This process was repeated for all of the remaining rows in movies.

At the end of the process, the movie_ratings DataFrame is created, which contains the two columns from movies (**movie_id** and **title**) and the three other colums from ratings (**user_id**, **rating**, and **timestamp**).

- **movie_id** 1 and its **title** are listed 452 times, next to the **user_id**, **rating**, and **timestamp** for each of the 452 matching ratings.
- **movie_id** 2 and its **title** are listed 131 times, next to the **user_id**, **rating**, and **timestamp** for each of the 131 matching ratings.
- And so on, for every movie in the dataset.

In [38]:
movie_ratings.loc[movie_ratings.movie_id == 1, :]

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
447,1,Toy Story (1995),895,4,879437950
448,1,Toy Story (1995),747,5,888639138
449,1,Toy Story (1995),786,4,882841828
450,1,Toy Story (1995),800,4,887646283


In [40]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 4)
(100000, 5)


Notice the shapes of the three DataFrames:

- There are 1682 rows in the movies DataFrame.
- There are 100000 rows in the ratings DataFrame.
- The `merge` function resulted in a movie_ratings DataFrame with 100000 rows, because every row from ratings matched a row from movies.
- The movie_ratings DataFrame has 5 columns, namely the 2 columns from movies, plus the 4 columns from ratings, minus the 1 column in common.

By default, the `merge` function joins the DataFrames using all column names that are in common (**movie_id**, in this case). The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge.html) explains how you can override this behavior.

**What if the columns you want to join on don't have the same name?**

In [42]:
movies.columns = ['m_id', 'title']

In [43]:
movies.columns

Index(['m_id', 'title'], dtype='object')

In [45]:
pd.merge(movies, ratings, left_on = 'm_id', right_on = 'movie_id')

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),308,1,4,887736532
1,1,Toy Story (1995),287,1,5,875334088
2,1,Toy Story (1995),148,1,4,877019411
3,1,Toy Story (1995),280,1,4,891700426
4,1,Toy Story (1995),66,1,3,883601324
...,...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1678,1,889289570
99996,1679,B. Monkey (1998),863,1679,3,889289491
99997,1680,Sliding Doors (1998),863,1680,2,889289570
99998,1681,You So Crazy (1994),896,1681,3,887160722


#### What if you want to join on one index?

In [47]:
ratings_index = ratings.set_index('movie_id')
ratings_index

,user_id,rating,timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596
...,...,...,...
476,880,3,880175444
204,716,5,879795543
1090,276,1,874795795


In [48]:
pd.merge(movies, ratings_index, left_on = 'm_id', right_index = True)

,m_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
0,1,Toy Story (1995),287,5,875334088
0,1,Toy Story (1995),148,4,877019411
0,1,Toy Story (1995),280,4,891700426
0,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
1677,1678,Mat' i syn (1997),863,1,889289570
1678,1679,B. Monkey (1998),863,3,889289491
1679,1680,Sliding Doors (1998),863,2,889289570
1680,1681,You So Crazy (1994),896,3,887160722


#### What if you want to join on two indexes?

In [49]:
movies_index = movies.set_index('m_id')
movies_index.head()

,title
m_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [50]:
pd.merge(movies_index, ratings_index, left_index = True, right_index = True)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


### Types of join

There are actually four types of joins supported by the Pandas `merge` function. Here's how they are described by the documentation:

- **inner:** use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
- **outer:** use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
- **left:** use only keys from left frame, similar to a SQL left outer join; preserve key order
- **right:** use only keys from right frame, similar to a SQL right outer join; preserve key order

The default is the "inner join", which was used when creating the movie_ratings DataFrame.

It's easiest to understand the different types by looking at some simple examples:

![join](https://www.dofactory.com/img/sql/sql-joins.png)

In [51]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})
A

,color,num
0,green,1
1,yellow,2
2,red,3


In [52]:
B = pd.DataFrame({'color': ['green', 'yellow', 'pink'], 'size':['S', 'M', 'L']})
B

,color,size
0,green,S
1,yellow,M
2,pink,L


#### Inner join

Only include observations found in both A and B:

In [53]:
pd.merge(A, B, how = 'inner')

,color,num,size
0,green,1,S
1,yellow,2,M


In [66]:
pd.merge(A, B, left_index = True, right_index = True, how = 'inner')

,color_x,num,color_y,size
0,green,1,green,S
1,yellow,2,yellow,M
2,red,3,pink,L


#### Outer join

Include observations found in either A or B:

In [55]:
pd.merge(A, B, how = 'outer')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,red,3.0,NaN
3,pink,NaN,L


#### Left join

Include all observations found in A:

In [56]:
pd.merge(A, B, how = 'left')

,color,num,size
0,green,1,S
1,yellow,2,M
2,red,3,NaN


#### Right join

Include all observations found in B:

In [65]:
pd.merge(A, B, how = 'right')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,pink,NaN,L


In [64]:
pd.merge(A, B, left_index = True, right_index = True, how = 'right')

,color_x,num,color_y,size
0,green,1,green,S
1,yellow,2,yellow,M
2,red,3,pink,L


In [63]:
# con el ejemplo de movies index y ratings index
pd.merge(movies_index, ratings_index, left_index = True, right_index = True, how = "outer")

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


# Join

Join columns of another DataFrame.

Join columns with other DataFrame either on index or on a key column. Efficiently join multiple DataFrame objects by index at once by passing a list.

If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.

While `merge` is a module function, `.join` is an object function that lives on your DataFrame. This enables you to specify only one DataFrame, which will join the DataFrame you call `.join` on.

Under the hood, `.join` uses `merge`

In [67]:
B.join(A, lsuffix = "_B", rsuffix = "_A")

,color_B,size,color_A,num
0,green,S,green,1
1,yellow,M,yellow,2
2,pink,L,red,3


In [68]:
#con las películas
display(movies.head())
ratings.head()

,m_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [69]:
movies.join(ratings)

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),196,242,3,881250949
1,2,GoldenEye (1995),186,302,3,891717742
2,3,Four Rooms (1995),22,377,1,878887116
3,4,Get Shorty (1995),244,51,2,880606923
4,5,Copycat (1995),166,346,1,886397596
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),303,94,3,879485318
1678,1679,B. Monkey (1998),99,50,5,885679998
1679,1680,Sliding Doors (1998),306,14,5,876503995
1680,1681,You So Crazy (1994),92,709,2,875654590


In [75]:
ratings.iloc[[0]]

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949


In [74]:
movies.iloc[[0]]

,m_id,title
0,1,Toy Story (1995)


In [70]:
ratings.join(movies)

,user_id,movie_id,rating,timestamp,m_id,title
0,196,242,3,881250949,1.0,Toy Story (1995)
1,186,302,3,891717742,2.0,GoldenEye (1995)
2,22,377,1,878887116,3.0,Four Rooms (1995)
3,244,51,2,880606923,4.0,Get Shorty (1995)
4,166,346,1,886397596,5.0,Copycat (1995)
...,...,...,...,...,...,...
99995,880,476,3,880175444,NaN,NaN
99996,716,204,5,879795543,NaN,NaN
99997,276,1090,1,874795795,NaN,NaN
99998,13,225,2,882399156,NaN,NaN


In [76]:
#si tienen mismo index, igual que al hacer pd.merge(movies_index, ratings_index, left_index=True, right_index=True)
movies_index.join(ratings_index)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


In [77]:
pd.merge(movies_index, ratings_index, left_index=True, right_index=True)

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


# Concatenating Along an Axis

# Concat

Concatenate pandas objects along a particular axis with optional set logic along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis, which may be useful if the labels are the same (or overlapping) on the passed axis number.

Concatenation is a bit different from the merging techniques you saw above. With merging, you can expect the resulting dataset to have rows from the parent datasets mixed in together, often based on some commonality. Depending on the type of merge, you might also lose rows that don’t have matches in the other dataset.

With concatenation, your datasets are just stitched together along an axis — either the row axis or column axis. Visually, a concatenation with no parameters along rows would look like this:

![](https://files.realpython.com/media/concat_axis0.2ec65b5f72bc.png)

To implement this in code, you’ll use concat() and pass it a list of DataFrames that you want to concatenate. 

**objs**: This parameter takes any sequence (typically a list) of Series or DataFrame objects to be concatenated. You can also provide a dictionary. In this case, the keys will be used for the keys options.

**axis**: Like in the other techniques, this represents the axis you will concatenate along. The default value is 0, which concatenates along the index (or row axis), while 1 concatenates along columns (vertically). You can also use the string values index or columns.

**join**: This is similar to the how parameter in the other techniques, but it only accepts the values inner or outer. The default value is outer, which preserves data, while inner would eliminate data that does not have a match in the other dataset.

In [84]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ["a", "b", "c"], columns = ["one", "three"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ["a", "c"], columns = ["three", "four"])

In [85]:
df1

,one,three
a,0,1
b,2,3
c,4,5


In [86]:
df2

,three,four
a,5,6
c,7,8


In [87]:
pd.concat([df1, df2], axis = 1)

,one,three,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [91]:
pd.concat([df1, df2], join = 'outer') # axis 0 por defecto

,one,three,four
a,0.0,1,NaN
b,2.0,3,NaN
c,4.0,5,NaN
a,NaN,5,6.0
c,NaN,7,8.0


In [92]:
pd.concat([df1, df2], join = 'inner')

,three
a,1
b,3
c,5
a,5
c,7


A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the `keys` argument.

In [93]:
pd.concat([df1, df2], axis = 1, keys = ["level1", "level2"])

level1       level2     
     one three  three four
a      0     1    5.0  6.0
b      2     3    NaN  NaN
c      4     5    7.0  8.0

If you pass a dict of objects instead of a list, the dict's keys will be used for the keys option: 


In [94]:
pd.concat({"level1": df1, "level2": df2}, axis = 1)

level1       level2     
     one three  three four
a      0     1    5.0  6.0
b      2     3    NaN  NaN
c      4     5    7.0  8.0

In [98]:
#usando movies y ratings vemos que para este problema concat no sirve
pd.concat([movies, ratings], axis = 1, keys = ["movies", "ratings"], join = "inner")

movies                                            ratings           \
       m_id                                      title user_id movie_id   
0         1                           Toy Story (1995)     196      242   
1         2                           GoldenEye (1995)     186      302   
2         3                          Four Rooms (1995)      22      377   
3         4                          Get Shorty (1995)     244       51   
4         5                             Copycat (1995)     166      346   
...     ...                                        ...     ...      ...   
1677   1678                          Mat' i syn (1997)     303       94   
1678   1679                           B. Monkey (1998)      99       50   
1679   1680                       Sliding Doors (1998)     306       14   
1680   1681                        You So Crazy (1994)      92      709   
1681   1682  Scream of Stone (Schrei aus Stein) (1991)     227      295   

                        
     rating  timestamp  
0         3  881250949  
1         3  891717742  
2         1  878887116  
3         2  880606923  
4         1  886397596  
...     ...        ...  
1677      3  879485318  
1678      5  885679998  
1679      5  876503995  
1680      2  875654590  
1681      5  879035387  

[1682 rows x 6 columns]

# Append

Because direct array concatenation is so common, ``Series`` and ``DataFrame`` objects have an ``append`` method that can accomplish the same thing in fewer keystrokes.

Append is the specific case `(axis=0, join='outer')` of concat

For example, rather than calling ``pd.concat([df1, df2])``, you can simply call ``df1.append(df2)``:

In [99]:
df_a = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df2_a = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))

In [100]:
df_a

,A,B
0,1,2
1,3,4


In [101]:
df2_a

,A,B
0,5,6
1,7,8


In [102]:
df_a.append(df2_a)

,A,B
0,1,2
1,3,4
0,5,6
1,7,8


In [103]:
%timeit df_a.append(df2_a)

225 µs ± 5.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [104]:
%timeit pd.concat([df_a, df2_a])

218 µs ± 5.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [105]:
# concat también tiene el parámetro ignore_index = True
df_a.append(df2_a, ignore_index = True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8


In [108]:
# esto sería lo mismo que usar ignore_index
df_a.append(df2_a).reset_index(drop = True)

,A,B
0,1,2
1,3,4
2,5,6
3,7,8
